In [15]:
from dotenv import load_dotenv
from pathlib import Path
import os
import sys
import json
import requests
import openai


env_path = Path('../..') / '.env'
load_dotenv(dotenv_path=env_path)

path = os.getcwd()
sys.path.append(path)

ISCRAPER_API_KEY = os.environ.get('ISCRAPER_API_KEY')
openai.api_key = os.environ.get('OPENAI_KEY')
PROFILE_DETAILS_URL = "https://api.proapis.com/iscraper/v4/profile-details"

        

In [16]:
def call_iscraper(linkedin_url: str):
    linkedin_id = linkedin_url.split("/in/")[1]
    payload = json.dumps(
        {
            "profile_id": linkedin_id,
            "profile_type": "personal",
            "network_info": True,
        }
    )
    headers = {"X-API-KEY": ISCRAPER_API_KEY, "Content-Type": "application/json"}

    response = requests.request(
        "POST", PROFILE_DETAILS_URL, headers=headers, data=payload
    )
    data = json.loads(response.text)

    return data

In [17]:
# Taken from src/ml/openai_wrappers.py
from typing import Optional, Union


CURRENT_OPENAI_DAVINCI_MODEL = "text-davinci-003"
CURRENT_OPENAI_CHAT_GPT_MODEL = "gpt-3.5-turbo"
CURRENT_OPENAI_LATEST_GPT_MODEL = "gpt-4"
DEFAULT_SUFFIX = None
DEFAULT_MAX_TOKENS = 16
DEFAULT_TEMPERATURE = 1
DEFAULT_TOP_P = 1
DEFAULT_N = 1
DEFAULT_FREQUENCY_PENALTY = 0
DEFAULT_STOP = None

def wrapped_chat_gpt_completion(
    messages: list,
    history: Optional[list] = [],
    max_tokens: Optional[int] = DEFAULT_MAX_TOKENS,
    temperature: Optional[float] = DEFAULT_TEMPERATURE,
    top_p: Optional[float] = DEFAULT_TOP_P,
    n: Optional[int] = DEFAULT_N,
    frequency_penalty: Optional[float] = DEFAULT_FREQUENCY_PENALTY,
    stop: Optional[Union[str, list]] = DEFAULT_STOP,
) -> str:
    """
    Generates a completion using the GPT-3.5-turbo model.

    messages needs to be in the format:
    [
        {
            "role": "user",
            "content": "Hello, how are you?"
        },
        {
            "role": "assistant",
            "content": "I am doing well, how about you?"
        }
        ...
    ]
    """
    if history:
        messages = history + messages

    response = openai.ChatCompletion.create(
        model=CURRENT_OPENAI_CHAT_GPT_MODEL,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        n=n,
        frequency_penalty=frequency_penalty,
        stop=stop,
    )
    if response is None or response["choices"] is None or len(response["choices"]) == 0:
        return ""

    choices = response["choices"]
    top_choice = choices[0]
    preview = top_choice["message"]["content"].strip()

    messages = messages + [{"role": "assistant", "content": preview}]
    return messages, preview


In [47]:
def get_impact_at_company(linkedin_url: str):

    # Assemble the job report
    data = call_iscraper(linkedin_url)
    job_report = ""

    for experience in data['position_groups']:
        company = experience['company']['name']
        job_report += f"Company: {company}\n"

        for position in experience['profile_positions']:
            title = position['title']
            description = position['description']
            start_year = position['date']['start']['year']
            end_year = position['date']['end']['year'] or "Current"

            job_report += f"{title} ({start_year}-{end_year}): {description}\n\n"

    job_report += f"McDonald's"
    job_report += f"Burger Flipper (2012 - 2014): Flipped burgers and made fries."

    # print(job_report)

    # Generate the impact
    messages = [
        {"role": "user", "content": job_report},
        {"role": "user", "content": "Given the job report above, describe the most impressive and impactful roles in the person's career. Please limit to 3 to 4 sentences"},
        # {"role": "user", "content": "I'm looking for people who can organize large events. Please use the job report above to relate this person's skills and experience to my request."},
    ]
    _, preview = wrapped_chat_gpt_completion(
        messages,
        max_tokens=500,
    )

    print(preview)
    # return preview



In [48]:
get_impact_at_company('linkedin.com/in/jacqxu00')

Based on the job history provided, the individual has experience organizing and leading events, specifically in the areas of design, innovation, and entrepreneurship. They have organized a nationwide conference with speakers from prestigious companies such as IDEO and Google, showcasing their ability to plan and execute large-scale events. Additionally, they have led workshops on the theme of inclusion, diversity, equity, and accessibility in design, demonstrating their expertise in event organization and facilitation.

Their role as the co-founder and co-president of Princeton ResInDe involved delivering product design and strategy through user research, UI/UX design, and digital implementation. This experience likely required them to coordinate and manage various tasks and resources, showcasing their organizational skills.

Furthermore, as a design associate for the Princeton Tiger Challenge, they mentored social impact teams and conducted user research, indicating their ability to s